In [2]:
using Printf
using DataFrames
using CSV
using LinearAlgebra
# using DataStructures

include("RLalgo.jl")

extract_state (generic function with 1 method)

### initialization of database

In [34]:
A = reshape(collect(1:100),(10,10))

10×10 Matrix{Int64}:
  1  11  21  31  41  51  61  71  81   91
  2  12  22  32  42  52  62  72  82   92
  3  13  23  33  43  53  63  73  83   93
  4  14  24  34  44  54  64  74  84   94
  5  15  25  35  45  55  65  75  85   95
  6  16  26  36  46  56  66  76  86   96
  7  17  27  37  47  57  67  77  87   97
  8  18  28  38  48  58  68  78  88   98
  9  19  29  39  49  59  69  79  89   99
 10  20  30  40  50  60  70  80  90  100

In [1]:
infile = "data/small.csv"
df = CSV.File(infile) |> DataFrame
# data_mat = Matrix(df);

x = [mod(df.s[j], 10)!=0 ? mod(df.s[j], 10) : 10 for j in 1:size(df,1)] 
y = [mod(df.s[j], 10)!=0 ? df.s[j] ÷ 10 + 1 : df.s[j] ÷ 10  for j in 1:size(df,1)] 

df = insertcols!(df, 2, :s_i => [x[i] for i in 1:size(df,1)])
df = insertcols!(df, 3, :s_j => [y[j] for j in 1:size(df,1)])

xp = [mod(df.sp[j], 10)!=0 ? mod(df.sp[j], 10) : 10 for j in 1:size(df,1)] 
yp = [mod(df.sp[j], 10)!=0 ? df.sp[j] ÷ 10 + 1 : df.sp[j] ÷ 10  for j in 1:size(df,1)] 

df = insertcols!(df, 7, :sp_i => [xp[i] for i in 1:size(df,1)])
df = insertcols!(df, 8, :sp_j => [yp[j] for j in 1:size(df,1)])

LoadError: UndefVarError: `CSV` not defined

In [36]:
idx = findall(df.r .> 0)
k   = rand([2,19,5,6,10])

19

### Gradient Q-learning (not deep, w/o experience replay)

In [37]:
# S = [[x,y] for x in 1:10, y in 1:10]   # FIXME: need to reshape this 
S = [i for i in 1:100]
A = [1,2,3,4]
γ = 0.95
T = NaN
R = NaN
TR = NaN

prob = MDP(γ,S,A,T,R,TR)

MDP(0.95, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [1, 2, 3, 4], NaN, NaN, NaN)

In [38]:
α = 0.5 # learning rate 
Q = zeros((length(S), length(A)))
N = zeros((length(S), length(A)))
model = SarsaLambda(S, A, γ, Q, N, α, 0.9, nothing)

SarsaLambda([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [1, 2, 3, 4], 0.95, [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], 0.5, 0.9, nothing)

In [39]:
train_offline(prob, model, df, 1e6)

In [29]:
model.Q

100×4 Matrix{Float64}:
 0.911615  0.844495   0.897917  0.960065
 0.859474  0.994003   0.870511  0.95927
 0.983386  1.67677    0.872321  1.17135
 1.93056   6.92699    3.49316   1.86597
 9.0034    3.15063    2.39925   3.49515
 2.49314   2.90701    4.19951   3.87671
 3.2575    4.72762   15.9033    8.04563
 5.03377   3.33238    3.5345    3.17158
 3.22592   2.87667    3.29735   2.22127
 2.9573    2.70889    2.48065   2.86824
 0.993548  1.27698    0.870038  0.943073
 0.895505  1.62467    1.71975   0.775533
 1.47563   5.11633    1.62095   1.01729
 ⋮                              
 1.03561   1.11031    0.827323  1.06439
 0.864282  0.854279   0.925336  0.813059
 4.43586   3.30255    2.8142    1.68264
 2.20963   2.9631     1.70278   2.82759
 2.68856   3.07944    1.91795   2.59893
 2.18365   2.28366    2.04682   1.40266
 0.935633  1.17189    1.35341   1.32271
 0.881991  1.07763    0.810712  0.940565
 0.937076  0.80904    0.855485  0.820364
 0.757782  0.973041   0.763696  0.873289
 0.84356   0.9292

In [40]:
A = reshape(collect(1:100),(10,10))

10×10 Matrix{Int64}:
  1  11  21  31  41  51  61  71  81   91
  2  12  22  32  42  52  62  72  82   92
  3  13  23  33  43  53  63  73  83   93
  4  14  24  34  44  54  64  74  84   94
  5  15  25  35  45  55  65  75  85   95
  6  16  26  36  46  56  66  76  86   96
  7  17  27  37  47  57  67  77  87   97
  8  18  28  38  48  58  68  78  88   98
  9  19  29  39  49  59  69  79  89   99
 10  20  30  40  50  60  70  80  90  100

In [41]:
a_opt = [findmax(model.Q[x, :])[2] for x in 1:100]
a_opt2 = reshape(a_opt, (10,10))

10×10 Matrix{Int64}:
 1  2  3  2  2  2  1  3  4  4
 3  2  4  4  4  3  3  3  4  3
 2  4  2  1  2  4  1  3  1  3
 3  1  1  3  3  2  1  4  3  1
 2  2  2  2  2  4  4  4  1  1
 3  2  3  3  4  2  4  4  4  1
 1  2  2  4  4  4  2  3  1  4
 4  3  2  1  2  4  1  2  2  1
 2  3  2  1  2  2  3  1  4  4
 2  1  3  1  2  4  4  1  3  2

In [42]:
file = open("small.policy", "w")

# Write each element of the vector to the file on a new line
for element in a_opt
    println(file, element)
end

# Close the file
close(file)

In [ ]:
train_offline(prob, model, df, 10000) 